In [2]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [432]:
def parse_joint_QC_CC(date):
    df = pd.read_csv(f'~/measurements/{date}.csv', sep = ';')
    CC_and_QC = {}
    for CC_or_QC in ['CC', 'QC']:
        frame = pd.DataFrame()
        new_date = '.'.join(date[i:i+2] for i in range(0,4,2)) + '.' + date[4:]
        frame['date'] = [new_date for k in range(3)]
        data = df[[k for k in df.columns if k.startswith(CC_or_QC + '_')]].transpose().reset_index().drop('index',axis=1)
        frame = pd.concat([frame, data], axis = 1, ignore_index = False)
        #convert column names to str
        columns = df[CC_or_QC].to_list()
        new_columns = map(str, ['date'] + df[CC_or_QC].to_list())
        
        

        CC_and_QC[CC_or_QC] = pd.DataFrame(frame.values, columns = new_columns)
        CC_and_QC[CC_or_QC].dropna(axis = 1, inplace = True)
        CC_and_QC[CC_or_QC]['date'] =  pd.to_datetime(CC_and_QC[CC_or_QC]['date'], format= '%d.%m.%Y')
        if '0.0' in CC_and_QC[CC_or_QC].columns.to_list():
            CC_and_QC[CC_or_QC].rename(columns={'0.0': '0'}, inplace=True, errors='raise')

            
    return CC_and_QC

In [423]:
def pl4(x, A, B, C, D):
    """4PL logistic equation."""
    return ((A - D) / (1.0 + ((x / C) ** B))) + D


def pl5(x, A, B, C, D, E):
    """5PL logistic equation."""
    return ((A - D) / (1.0 + ((x / C) ** B) ** E)) + D


def revers_pl4(y, A, B, C, D):
    return C*(((A - y) / (y - D)) ** (1.0 / B))

In [4]:
def plot(dataframe, fun, x, ):
    popt, pcov = curve_fit(f = fun, xdata = dataframe[x], ydata = dataframe[y])
    df['fitted_y'] = fun(dataframe[x], *popt)

    if fun == pl4:
        label_text = '{} A=%5.3f, B=%5.3f, C=%5.3f, D=%5.3f'.format('fit 4PL')
        st.write(pd.DataFrame([popt], columns=['A', 'B', 'C', 'D'], index=[y]))
    
    xs = np.arange(dataframe[x].min(),dataframe[x].max(), 0.2)
    ys = fun(xs, *popt)
    plt.plot( xs, ys, label=label_text % tuple(popt))

    try:
        plt.errorbar(dataframe[x], dataframe[y], dataframe[err], fmt='o', label=f'data {y}')
    except KeyError:
        plt.plot(dataframe[x], dataframe[y], 'o', label=f'data {y}')
    plt.xscale("log")
    print(popt, pcov)

In [5]:
def fit_weighted(y, fun, conc, std):
#Weighted fit
    popt, pcov = curve_fit(f = fun, xdata = conc, ydata = y, sigma = std)
    return popt

def fit(y, fun, conc):
    popt, pcov = curve_fit(f = fun, xdata = conc, ydata = y)
    return popt

In [6]:
def pretify_df(df):
    
    df = df.transpose()
    df = df.reset_index()
    df.loc[0,'index'] = 'conc'
    df.columns = df.iloc[0,:]
    df=df.drop([0])
    df = df.set_index('conc')
    return df

In [256]:
def fit_CC(df1,
           drop_0 = 1, #if to use background ('0') level for fitting
           fun = pl4,
           weighted_fit = 1, #weighted or regular fitting
           mean_or_median = 'mean' #one may fit to median values
          ):
    
## Funnction fits parametric regression to experimental data 
    
    #Convert date to DateTime
    df = df1.copy()
    df['date'] =  pd.to_datetime(df['date'], format= '%d.%m.%Y')
    df['date'] = df['date'].dt.strftime('%d-%m-%Y')
    
    conc_cols = df.columns.drop('date')
    df[conc_cols] = df[conc_cols].astype(float)

    #Save background level
    
    blank = df[['date','0']].groupby('date', as_index = False).mean()
    blank = pretify_df(blank)
    
    #Drop backgrounnd if it is not used for calibration curve fitting
    if drop_0 == 1:
        df = df.drop('0', axis = 1)
    conc = df.columns.transpose().drop('date').astype(float)
            
    df_mean = df.groupby('date', as_index = False).mean()
    df_median = df.groupby('date', as_index = False).median()
    df_std = df.groupby('date').std().reset_index()
       
    df_mean = pretify_df(df_mean)
    df_median = pretify_df(df_median)
    df_std = pretify_df(df_std)
             
    params = {}
    
    if mean_or_median == 'mean':
        fit_to = df_mean.copy()
    else:
        fit_to = df_median.copy()
        
        
    for column in fit_to.columns:
        if weighted_fit == 1:
            params[column] = fit_weighted(fit_to[column], fun, conc, df_std[column])
        else:
            params[column] = fit(fit_to[column], fun, conc)        
    popt = pd.DataFrame.from_dict(params)
    
    return popt

In [272]:
def calc_metrics(df2, popt):
    
    #Calculate parametrs RE_mean (relative error of the mean value, accuracy),
    #CV, Tot (total error), RE (relative error of individual values),
    #Rel_MSE (should be the function to assess goodness of fit)
    
    df = df2.copy()
    
    #Convert date to DateTime
    df['date'] =  pd.to_datetime(df['date'], format='%d.%m.%Y')
    df['date'] = df['date'].dt.strftime('%d-%m-%Y')
    
    conc_cols = df.columns.drop('date')
    df[conc_cols] = df[conc_cols].astype(float)
    conc = df.columns.transpose().drop('date').astype(float)
        
    df_mean = df.groupby('date', as_index = False).mean()
    df_std = df.groupby('date').std().reset_index()
       
    df_mean = pretify_df(df_mean)
    df_std = pretify_df(df_std)
    
    #Backcalculate concentrations based on calibration curve
    calc_vals = {}
    for column in popt.columns:
        calc_vals[column] = df[df.date == column].drop('date', axis = 1).\
        applymap(lambda x: revers_pl4(x, *popt[column].values))
        calc_vals[column]['date'] = column
        
    df_calc_vals = pd.concat(calc_vals.values())
    
    df_calc_vals['date']=pd.to_datetime(df_calc_vals['date'], format='%d-%m-%Y')
    #Sort dates to pretify RE output
    df_calc_vals = df_calc_vals.sort_values(by = 'date', axis = 0, ascending = False)

    df_calc_vals_mean = df_calc_vals.groupby('date').mean()
    df_calc_vals_std = df_calc_vals.groupby('date').std()

    #Calculete CV
    CV = df_calc_vals_std/df_calc_vals_mean*100
    CV = CV.sort_index(ascending = False)
    
    #Access accuracy of mean values (on concentration level)
    RE_mean = df_calc_vals_mean.apply(lambda x: abs(1 - x/conc)*100, axis = 1).sort_index(ascending = False)
    
    #Total error
    Tot = CV+RE_mean
    Tot = Tot.sort_index(ascending = False)
    
    #Access individual concentration point accuracy
    df_calc_vals.set_index([df_calc_vals.index, 'date'], inplace = True)
    RE = df_calc_vals.apply(lambda x: 100* np.abs(x/conc-1), axis = 1)    
    
    Rel_MSE = df_calc_vals_mean.apply(lambda x: (x/conc - 1)**2, axis = 1).mean(axis = 1).sort_index(ascending = False)
    
    return RE_mean, CV, Tot, RE, Rel_MSE

In [273]:
def less(x, val):
    if x<=val:
        return 1
    else:
        return 0

def analyze_metrics(df, mode,  ULOQ, LLOQ):
    
# Function calculates if calibration cureve or QC parametrs meet acceptance conditions

# Input - either RE_mean, or CV, or Tot, or RE,
# In initial df index is dates, columns are concentrations 
# The output is df with number of standards met accuracy criteria per concentration level at each date
# '% standarts with acceptable CV' - percent of acceptable probes at each date
# 'all levels ok' indicates if every level contains at least one acceptable probe
# 'Accept CC run?' finally indicates if calibration curve is acceptable for validation run
    
    if (mode == 'QC_acc') | (mode == 'QC_CV') | (mode == 'CC_acc'):
        inner_treshold = 20
        LOQ_threshold = 25
    elif mode == 'Tot_err':
        inner_treshold = 30
        LOQ_threshold = 40
        
    inner_levels = []
    for conc in df.columns.to_list():
        if ((float(conc)<ULOQ) & (float(conc)>LLOQ)):
            inner_levels.append(conc)
            
    upper = df[[str(ULOQ)]].applymap(lambda x: less(x, LOQ_threshold))
    inner = df[inner_levels].applymap(lambda x: less(x, inner_treshold))
    lower = df[[str(LLOQ)]].applymap(lambda x: less(x, LOQ_threshold))
    res = pd.concat([upper,inner,lower], sort = True, axis = 1)
    
    replicate_number = len(res)/len(res.reset_index()['date'].unique())
    res = res.groupby('date').sum()
    
    if mode == 'CC_acc':
        res['% standarts with acceptable CV']= res.sum(axis = 1)/(replicate_number*len(res.columns))*100
        res['all levels ok'] = res.applymap(lambda x: x>0).all(axis = 1)
        res['Accept CC run?'] = np.where((res['% standarts with acceptable CV'] >=75) &\
                                      (res['all levels ok'] == True), 'YES', 'no')
    else:
        res['all levels ok'] = res.applymap(lambda x: x>0).all(axis = 1)
        res[mode + ' ACCEPTABLE?'] = np.where(res['all levels ok'] == True, 'YES', 'no')
        
        
    return res

In [10]:
def summary(dfs, ULOQ, LLOQ):
    
# Returns the summary with decision about acceptance of CC or QC run

    if len(dfs) == 3:
    #Returns summary for QC run    
    # input df list should be [RE_mean, CV, Tot]
    
        acc_summary = analyze_metrics(dfs[0], 'QC_acc',  ULOQ, LLOQ)[['QC_acc ACCEPTABLE?']]
        CV_summary = analyze_metrics(dfs[1], 'QC_CV',  ULOQ, LLOQ)[['QC_CV ACCEPTABLE?']]
        Tot_err_summary = analyze_metrics(dfs[2], 'Tot_err',  ULOQ, LLOQ)[['Tot_err ACCEPTABLE?']]
        summary = pd.concat([acc_summary, CV_summary, Tot_err_summary], axis = 1)
        summary['Accept A&P run?'] = np.where(((summary['QC_acc ACCEPTABLE?'] == 'YES')
                                              & (summary['QC_CV ACCEPTABLE?'] == 'YES')
                                              & (summary['Tot_err ACCEPTABLE?'] == 'YES')), 'YES', 'no')
        
    elif len(dfs) == 1:    
    #Returns summary for CC run
    # input df list should be [RE]

        summary = analyze_metrics(dfs[0], 'CC_acc',  ULOQ, LLOQ)
    return summary

In [11]:
def pipeline(dfs, ULOQ, LLOQ):
     
    # fit calibration curve
    popt = fit_CC(dfs[0], drop_0 = 1, fun = pl4, weighted_fit = 1, mean_or_median = 'mean')
    
    # Calculate metrics for calibration curve (CC)
    _, _, _, RE_CC, Rel_MSE = calc_metrics(dfs[0], popt)
    CC_summary = analyze_metrics(RE_CC, 'CC_acc',  ULOQ, LLOQ)
    
    # Calculate metrics for QCs
    if len(dfs) == 2:
        RE_mean, CV, Tot, RE, _ = calc_metrics(dfs[1], popt)
        QC_summary = summary([RE_mean, CV, Tot], ULOQ, LLOQ)
        return CC_summary, QC_summary
    else:
        return CC_summary

In [209]:
df_CC = pd.read_csv('~/measurements/calibration_data_04122020.csv', sep = ';')
df_QC = pd.read_csv('~/measurements/QC_data_04122020.csv', sep = ';')

In [438]:
CC_list = []
QC_list = []
for k in ['04', '08', '11', '17', '23']:
    name = f'{k}122020'    
    CC_list.append(parse_joint_QC_CC(name)['CC'])
    QC_list.append(parse_joint_QC_CC(name)['QC'])
CC_data = pd.concat(CC_list)
QC_data = pd.concat(QC_list)

In [450]:
CC_data

,date,2500.0,1250.0,625.0,312.5,156.3,78.1,39.1,19.5,9.8,0
0,2020-12-04,1.2618,1.1734,1.0455,0.7606,0.5073,0.286,0.1661,0.1125,0.0946,0.0782
1,2020-12-04,1.3002,1.1952,1.0314,0.7899,0.5461,0.323,0.2025,0.1227,0.0968,0.0795
2,2020-12-04,1.2511,1.1325,1.0302,0.7835,0.5221,0.299,0.1841,0.1207,0.0952,0.0799
0,2020-12-08,1.2447,1.3105,1.0707,0.8941,0.6458,0.4073,0.2515,0.1515,0.1258,0.0882
1,2020-12-08,1.3378,1.2707,1.1128,0.8638,0.6128,0.3809,0.244,0.1662,0.1329,0.0929
2,2020-12-08,1.3042,1.2691,1.0818,0.9033,0.6372,0.4031,0.2692,0.1658,0.1317,0.0869
0,2020-12-11,1.468,1.4091,1.2482,0.9777,0.7103,0.4362,0.2357,0.1329,0.0936,0.0673
1,2020-12-11,1.3636,1.2959,1.1503,0.9149,0.6452,0.377,0.2186,0.1293,0.0914,0.067
2,2020-12-11,1.4013,1.3527,1.1877,0.9271,0.6375,0.3688,0.2021,0.1228,0.0869,0.0674
0,2020-12-17,2.2194,1.9152,1.5449,0.8884,0.4174,0.2157,0.1189,0.09,0.0816,0.0797


In [456]:
QC_data

,date,1000.0,666.7,444.4,296.3,197.5,131.7,87.8,58.5,39.0,0
0,2020-12-04,1.1175,1.1044,1.0054,0.8352,0.7033,0.4774,0.3757,0.2633,0.1981,0.0781
1,2020-12-04,1.1657,1.0985,0.9382,0.751,0.6164,0.4666,0.3523,0.2641,0.1792,0.0789
2,2020-12-04,1.1566,1.0333,0.9207,0.805,0.6037,0.4687,0.3477,0.2407,0.1903,0.0818
0,2020-12-08,1.1668,1.0469,0.9477,0.8821,0.7537,0.5408,0.3983,0.3085,0.2339,0.0887
1,2020-12-08,1.2018,1.0763,0.9999,0.8007,0.6714,0.5175,0.3894,0.2905,0.2249,0.0881
2,2020-12-08,1.2398,1.1192,0.9428,0.7889,0.6792,0.5283,0.4165,0.3106,0.2872,0.1053
0,2020-12-11,1.2084,1.1088,1.0088,0.8646,0.7026,0.5476,0.4691,0.2715,0.196,0.0674
1,2020-12-11,1.2823,1.1094,1.0368,0.8977,0.757,0.5739,0.4294,0.3002,0.2043,0.0669
2,2020-12-11,1.2924,1.2673,1.1374,0.9866,0.804,0.6146,0.4814,0.3189,0.2318,0.0674
0,2020-12-17,1.6716,1.4402,1.1294,0.7602,0.4857,0.293,0.1915,0.1383,0.0981,0.08


In [441]:
popt = fit_CC(CC_data,
              drop_0 = 1,
              fun = pl4,
              weighted_fit =0,
              mean_or_median = 'mean')

In [460]:
_, _, _, RE_CC, Rel_MSE = calc_metrics(CC_data, popt)
CC_summary = analyze_metrics(RE_CC, 'CC_acc',  2500.0, 39.1)
CC_summary

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


,2500.0,1250.0,625.0,312.5,156.3,78.1,39.1,% standarts with acceptable CV,all levels ok,Accept CC run?
date,,,,,,,,,,
2020-12-04,2,2,3,3,3,3,3,90.476190,True,YES
2020-12-08,1,0,3,3,3,3,3,76.190476,False,no
2020-12-11,1,1,2,3,3,3,3,76.190476,True,YES
2020-12-17,2,3,3,3,3,3,3,95.238095,True,YES
2020-12-23,1,3,3,3,2,3,1,76.190476,True,YES


In [458]:
RE_mean, CV, Tot, RE, _ = calc_metrics(QC_data, popt)
QC_summary = summary([RE_mean, CV, Tot], 1000.0, 197.5)
QC_summary

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


,QC_acc ACCEPTABLE?,QC_CV ACCEPTABLE?,Tot_err ACCEPTABLE?,Accept A&P run?
date,,,,
2020-12-04,YES,YES,no,no
2020-12-08,YES,YES,YES,YES
2020-12-11,YES,no,no,no
2020-12-17,YES,YES,YES,YES
2020-12-23,no,no,no,no


In [453]:
RE_mean

,1000.0,666.7,444.4,296.3,197.5,131.7,87.8,58.5,39.0,0
date,,,,,,,,,,
2020-12-23,18.576854,25.029186,20.802674,22.724266,29.867730,26.323149,20.412692,14.487610,33.993184,NaN
2020-12-17,0.010501,3.205217,1.974094,0.851345,10.343977,13.419523,18.262431,25.956297,33.383071,inf
2020-12-11,16.579179,10.533806,4.977279,2.347702,0.008985,1.130965,8.594677,5.144123,6.061299,inf
2020-12-08,0.716581,14.299301,12.813763,11.048620,3.407514,9.853176,10.120516,10.026956,0.354224,inf
2020-12-04,4.215712,15.955566,13.432799,9.340225,10.238544,3.683842,8.255154,5.264311,4.965514,inf


In [454]:
CV

,1000.0,666.7,444.4,296.3,197.5,131.7,87.8,58.5,39.0,0
date,,,,,,,,,,
2020-12-23,30.674712,23.952646,21.479179,22.832647,29.611324,14.019668,30.059744,42.812485,55.020281,NaN
2020-12-17,15.044814,11.711817,14.985129,13.432326,8.440505,11.698739,8.477217,1.892013,7.890559,2.043277
2020-12-11,18.310867,35.125995,19.981833,15.725132,11.594819,8.370122,7.586482,9.654156,10.963233,1.687856
2020-12-08,20.546952,13.584823,9.764096,14.024583,12.226828,3.400575,4.980053,5.311973,21.106343,NaN
2020-12-04,12.249297,14.493095,14.239435,10.902955,14.349536,1.684139,5.446259,6.735805,7.124223,18.978830


In [455]:
Tot

,1000.0,666.7,444.4,296.3,197.5,131.7,87.8,58.5,39.0,0
date,,,,,,,,,,
2020-12-23,49.251566,48.981831,42.281853,45.556913,59.479054,40.342817,50.472436,57.300095,89.013466,NaN
2020-12-17,15.055315,14.917034,16.959223,14.283671,18.784481,25.118262,26.739648,27.848310,41.273630,inf
2020-12-11,34.890045,45.659800,24.959112,18.072834,11.603804,9.501087,16.181159,14.798279,17.024532,inf
2020-12-08,21.263533,27.884124,22.577859,25.073202,15.634342,13.253751,15.100569,15.338929,21.460566,NaN
2020-12-04,16.465009,30.448661,27.672234,20.243180,24.588079,5.367981,13.701413,12.000116,12.089737,inf
